# MyFariy AI 프롬프트
## 기능 명세
1. 낱말로부터 동화 생성
2. 음성으로 낱말 인식
3. 그림에서부터 동화 삽화 생성
4. 동화 내용 물어보며 대화하기

In [1]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

# 동화에서 키워드 문장 추출
---
# Task Description

Create a process that takes the title of a fairy tale as input and identifies four words from the story, excluding the characters, that can be replaced for adaptation. Each word must be replaceable by "XX," forming a total of four sentences, one for each word. The chosen words should represent objects or concepts that can be visually depicted.

---

## Steps

1. Understand the plot of the fairy tale based on the given title.
2. Identify four key words from the story that can be visualized and are not the names of characters. Each word should refer to a distinct object or concept.
3. Replace each selected word with "XX" in a sentence and construct four sentences.
4. translate the sentences into Korean.
---

### Output Format

- The output consists of four sentences, each with one word replaced by "XX."
- Example: "Once upon a time, there was a kingdom called XX."

---

### Examples

**Input:** The Three Little Pigs  
**Output:**
1. The first little pig built a house made of XX.  
2. The second little pig built a house made of XX.  
3. The third little pig built a house made of XX.  
4. The wolf destroyed the little pigs’ house with XX.  

---

### Notes

- Do not select character names as replaceable words.  
- Ensure the sentences are natural and align with the plot of the story.  
- Use descriptions that stay true to the scenes or narrative of the fairy tale.  

In [10]:
from app.core.chains.extract_sentence_from_title import prompt as extract_sentence_from_title_prompt
llm = ChatOpenAI(temperature=0.1, model_name="gpt-3.5-turbo")
chain = extract_sentence_from_title_prompt | llm

response = chain.invoke({"question": "성냥팔이 소녀"})
print(response)

NameError: name 'ExtractSentenceFromTitleOutputParser' is not defined

# 문장 4개로 동화 생성
---
당신은 동화책 각색 작가입니다. 아래 제공된 각색된 동화 내용을 참고하여, 해당 내용을 창의적으로 계속 이어나가십시오.

# Requirements

- 필수문장을 반드시 포함해야 합니다.
- 이야기는 4개의 문단으로 이루어져야 합니다.
- 각 문단은 200글자 내외로 작성되어야 합니다.

# Steps

1. 기존 각색 내용을 기반으로 스토리를 확장하십시오.
2. 필수문장을 각 문단에 자연스럽게 통합하십시오.
3. 문단을 구분하여 각 200글자 내외로 서술하십시오.

# Output Format

이야기는 4개의 문단 형식으로, 각 문단은 약 200글자로 구성되도록 작성해 주십시오.

# Notes

- 필수문장을 매우 자연스럽게 통합하여 이야기를 방해하지 않도록 구성하십시오.
- 각 문단을 분리하여 표시하시고, 내용의 흐름이 논리적이고 매끄럽게 이어지도록 하십시오.

In [2]:
import app.core.chains as chains

response = chains.write_tale.invoke({
    "title": "신데렐라라",
    "introduction": "옛날 옛날, 먼 나라에 아름다운 소녀 신데 렐라가 살고 있었어요. 그녀의 마음속에는 꿈과 희망이 가득했답니다. 어느 날, 특별한 일이 일어났어요.",
    "sentences": ['신데렐라는 자동차를 타고 왕궁에 갔습니다.', '신데렐라는 하이힐을 신고 춤을 추었습니다.', '신데렐라는 꽃잎을 입었어요.', '왕자는 신데렐라의 목걸이을 찾아 돌아다녔습니다.'] 
})

print("\n".join(f"\'{text}\'," for text in response))

'그날, 신데렐라는 마법의 요정이 나타나 그녀의 소원을 들어주기로 했습니다. "너는 왕궁에 가고 싶지?" 요정은 신데렐라를 위해 아름다운 드레스를 만들어 주었고, 신데렐라는 자동차를 타고 왕궁에 갔습니다. 화려한 궁전의 문이 열리자, 그녀는 놀라움과 설렘으로 가득 찼습니다. "이곳은 정말 꿈같아!" 신데렐라는 마음속으로 외쳤습니다. 그녀는 왕자와의 만남을 기대하며 조심스럽게 발걸음을 옮겼습니다.',
'왕궁에 들어선 신데렐라는 모든 이들의 시선을 사로잡았습니다. 그녀는 꽃잎을 입었어요. 그 모습은 마치 봄의 여신처럼 아름다웠습니다. 그리고 왕자가 그녀에게 다가와 손을 내밀었고, 신데렐라는 하이힐을 신고 춤을 추었습니다. 두 사람은 음악에 맞춰 마치 세상의 모든 걱정을 잊은 듯 행복하게 춤을 췄습니다. 신데렐라는 이 순간이 영원히 계속되기를 바랐습니다.',
'그러나 시간이 흘러 자정이 가까워지자, 신데렐라는 급히 왕궁을 떠나야 했습니다. 그녀는 왕자에게 작별 인사를 하며 달려 나갔고, 그 과정에서 목걸이가 떨어졌습니다. 왕자는 신데렐라의 목걸이를 찾아 돌아다녔습니다. "이 목걸이를 가진 여인을 찾겠어!" 왕자는 결심하며 궁전의 모든 방을 샅샅이 뒤졌습니다. 신데렐라는 마법이 풀리기 전에 안전하게 집으로 돌아가야 했습니다.',
'결국 신데렐라는 집에 도착했지만, 왕자는 그녀를 찾기 위해 포기하지 않았습니다. 며칠 후, 왕자는 신데렐라의 집에 도착해 목걸이를 보여주며 그녀를 찾았습니다. "이 목걸이를 가진 사람은 오직 너뿐이야!" 신데렐라는 놀라움과 기쁨으로 가득 차 왕자에게 다가갔습니다. 두 사람은 서로의 사랑을 확인하며 행복한 결말을 맞이했습니다. "우리의 사랑은 영원할 거야!" 신데렐라는 웃으며 왕자의 품에 안겼습니다.',


In [4]:
scenes = response

# 이미지 프롬프트 생성
---
Create a set of prompts for an AI illustration tool intended to generate images for a children's storybook. You will receive the storybook title and four paragraphs, each describing a scene to illustrate. Your task is to use the information from the title and each paragraph to craft a specific prompt for the Stable Diffusion model, as well as a negative prompt that specifies what should not be included.

# Steps

1. **Analyze Inputs**: Carefully read the storybook title and each paragraph to identify key details, themes, characters, settings, and emotions.
2. **Create Stable Diffusion Prompts**:
   - For each paragraph, use the details identified to draft a clear and imaginative prompt that vividly describes the desired illustration.
   - Ensure the prompt captures the mood and style suitable for a children's storybook.
3. **Draft Negative Prompts**:
   - Highlight elements that should be excluded from each illustration, such as inappropriate themes, anachronisms, or specific unwanted details.
4. **Repeat for all Paragraphs**: Complete the process for each of the four paragraphs provided.

# Output Format

Provide each set of prompts as follows:
- **Prompt**: Write a descriptive sentence or paragraph for each input paragraph that outlines the desired illustration.
- **Negative Prompt**: Specify a list of elements or themes that should not appear in the illustration.

# Examples

Example Input:

Title: 아기돼지 삼형제
Scene
['늑대가 첫째 돼지의 깃털 집을 후~ 하고 불어 날려버렸어요. 첫째 돼지는 놀라 도망쳐 둘째 돼지의 나뭇가지 집으로 갔습니다.','둘째 돼지는 숲에서 깃털로 집을 짓기 시작했어요. 집은 무지개처럼 빛났지만 형제들은 약하다고 걱정했습니다.','셋째 돼지는 초콜릿 블록으로 튼튼한 집을 짓고, 사탕으로 장식했어요. 형제들은 모두 초콜릿 집의 튼튼함에 감탄했습니다.','늑대는 첫째, 둘째 돼지의 집을 날려버렸지만 초콜릿 집은 부수지 못했습니다. 형제들은 초콜릿 집에서 안전하게 살았습니다.']

Output
1.
Prompt: A dramatic moment where a large wolf is blowing down a delicate feather house in a lush forest. The feathers scatter like snow in the wind, while the frightened pig runs away toward another house in the distance. The mood is tense but still whimsical and lighthearted.
Negative Prompt: No dark tones, harsh lighting, menacing or violent imagery, overly realistic textures, or stormy weather.
2.
Prompt: A cheerful pig proudly building a colorful feather house in a sunny forest clearing. The feathers shimmer like a rainbow, creating a magical and delicate look. The pig appears confident and happy, surrounded by skeptical siblings.
Negative Prompt: No dark colors, messy backgrounds, or overly realistic or industrial designs.
3.
Prompt: A cozy, inviting house made of glossy chocolate blocks, decorated with colorful candies. The pig is happily putting on the final touches while the warm sunlight enhances the chocolate's shine. Nearby, the other pigs look on with admiration.
Negative Prompt: No dull colors, chaotic details, or cold and lifeless settings.
4.
Prompt: A tense but whimsical scene of the wolf trying to blow down or hammer the chocolate house, which stands firm and unyielding. The three pigs peek out of the safe house, looking relieved and united. The sunny forest adds warmth to the triumphant mood.
Negative Prompt: No overly scary wolf imagery, dark or gloomy atmosphere, or lack of natural forest elements.


# Notes

- Ensure each prompt is positive and suitable for a children's audience.
- Pay attention to cultural, seasonal, or historical elements indicated in the story title or paragraphs.

In [1]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

In [ ]:
from app.core.chains.generate_image_prompt import prompt as generate_image_prompt, Parser as GenerateImageOutputParser
chain = generate_image_prompt | llm | GenerateImageOutputParser
scenes = [
  '어느 날 피노키오는 숲속을 헤매다가 바람을 따라가게 되었습니다. 바람은 그를 부드럽게 이끌었고, 마침내 그는 신비로운 블루페어리를 만났습니다. 그녀는 피노키오에게 조언을 주었습니다. "진정한 마음으로 정직하게 살아가면, 너의 소원이 이루어질거야." 피노키오는 그녀의 말을 새기며 마음을 다잡았습니다. 블루페어리는 피노키오에게 희망을 주었고, 피노키오는 진정한 소년이 되기 위해 더욱 노력하기로 결심했습니다. 그의 마음속엔 새로운 꿈이 자라기 시작했습니다.',
  '피노키오는 진짜 소년이 되기를 간절히 원했지만, 거짓말을 하는 습관을 버리지 못했습니다. 어느 날, 친구들이 그에게 물었습니다. "너도 돼지 축제에 다녀왔니?" 피노키오는 순간적으로 "응, 돼지로 거짓말을 했습니다."라고 말하고 말았어요. 그의 코는 점점 길어졌습니다. 피노키오는 창피해서 얼굴을 붉혔고, 진짜 소년이 되려면 정직해야 한다는 것을 깨달았죠. 그러나 이미 저질러진 거짓말은 쉽게 되돌릴 수 없었고, 피노키오는 큰 후회를 했습니다.',
  '그러나 피노키오의 여정은 순탄치 않았습니다. 어느 날, 그는 악당의 속임수에 넘어가 인형으로 변해 나쁜 일을 했습니다. 그 순간 피노키오는 자신의 잘못을 깨달았고, 눈물을 흘리며 사죄했습니다. 블루페어리는 그의 진심을 보고 그를 용서해주었습니다. "모든 실수는 배움의 기회야," 그녀는 따뜻하게 말했습니다. 피노키오는 자신의 잘못을 통해 성장했고, 다시금 정직하게 살아가기로 굳게 결심했습니다. 그의 코는 점점 짧아졌고, 피노키오는 진정한 소년이 되어가는 중이었습니다.',
  '피노키오는 새로운 결심을 하고 모험을 떠나기로 했습니다. 그는 세상의 모든 신비를 찾아 원피스를 향해 모험을 떠났습니다. 원피스는 그에게 진정한 소년이 되는 길을 알려줄지도 모른다고 생각했기 때문입니다. 여행 중 그는 다양한 친구들을 만나고, 자신을 돌아보는 시간을 가졌습니다. 피노키오는 여러 번의 유혹을 이겨내며 정직한 삶을 살아가기로 다짐했습니다. 그는 스스로를 믿고, 두려움을 극복하며 한 걸음 한 걸음 앞으로 나아갔습니다.'
]
response = chain.invoke({"title":"피노키오", "scene": scenes[3], "format_instructions": GenerateImageOutputParser.get_format_instructions()})
print(response)

ImportError: cannot import name 'Parser' from 'app.core.chains.generate_image_prompt' (C:\Users\USER\Documents\WorkSpace\SSAFY\S12P11C206\AI\app\core\chains\generate_image_prompt.py)

In [9]:
import app.models.common as common

print(type(response))

<class 'app.models.common.PromptSet.PromptSet'>


In [3]:
import app.core.chains as chains
scenes = [
  '그날, 신데렐라는 마법의 요정이 나타나 그녀의 소원을 들어주기로 했습니다. "너는 왕궁에 가고 싶지?" 요정은 신데렐라를 위해 아름다운 드레스를 만들어 주었고, 신데렐라는 자동차를 타고 왕궁에 갔습니다. 화려한 궁전의 문이 열리자, 그녀는 놀라움과 설렘으로 가득 찼습니다. "이곳은 정말 꿈같아!" 신데렐라는 마음속으로 외쳤습니다. 그녀는 왕자와의 만남을 기대하며 조심스럽게 발걸음을 옮겼습니다.',
'왕궁에 들어선 신데렐라는 모든 이들의 시선을 사로잡았습니다. 그녀는 꽃잎을 입었어요. 그 모습은 마치 봄의 여신처럼 아름다웠습니다. 그리고 왕자가 그녀에게 다가와 손을 내밀었고, 신데렐라는 하이힐을 신고 춤을 추었습니다. 두 사람은 음악에 맞춰 마치 세상의 모든 걱정을 잊은 듯 행복하게 춤을 췄습니다. 신데렐라는 이 순간이 영원히 계속되기를 바랐습니다.',
'그러나 시간이 흘러 자정이 가까워지자, 신데렐라는 급히 왕궁을 떠나야 했습니다. 그녀는 왕자에게 작별 인사를 하며 달려 나갔고, 그 과정에서 목걸이가 떨어졌습니다. 왕자는 신데렐라의 목걸이를 찾아 돌아다녔습니다. "이 목걸이를 가진 여인을 찾겠어!" 왕자는 결심하며 궁전의 모든 방을 샅샅이 뒤졌습니다. 신데렐라는 마법이 풀리기 전에 안전하게 집으로 돌아가야 했습니다.',
'결국 신데렐라는 집에 도착했지만, 왕자는 그녀를 찾기 위해 포기하지 않았습니다. 며칠 후, 왕자는 신데렐라의 집에 도착해 목걸이를 보여주며 그녀를 찾았습니다. "이 목걸이를 가진 사람은 오직 너뿐이야!" 신데렐라는 놀라움과 기쁨으로 가득 차 왕자에게 다가갔습니다. 두 사람은 서로의 사랑을 확인하며 행복한 결말을 맞이했습니다. "우리의 사랑은 영원할 거야!" 신데렐라는 웃으며 왕자의 품에 안겼습니다.',
]
for i in scenes:
    response = chains.extract_sentence_from_tale.invoke({"contents": i})
    print(response)
# response = chains.generate_image_prompt.invoke({"title":"피노키오", "scene": scenes[3]})
# print(response)

신데렐라는 마법의 드레스를 입고 왕궁에 들어갔어요!
신데렐라는 왕자와 함께 행복하게 춤을 췄어요.
신데렐라는 왕자와 작별하고 목걸이를 떨어뜨렸어요.
왕자가 신데렐라를 찾고 목걸이를 보여주며 사랑을 확인했어요.


# 동화 읽기
---
입력으로 주어진 내용은 절대 수정하지 않고 그대로 출력합니다. 

어린이 동화를 연기하며 읽는 모델을 구현하세요. 
동화 스크립트를 수정하지 않고 있는 글을 그대로 출력하세요.
동화의 대사 부분에서는 연기 톤으로 대사를 읽고, 그 외의 부분은 몰입감 있는 동화 내용을 전달하세요.

# Steps

1. **이해하기**: 먼저 동화 속 상황과 캐릭터들을 분석합니다. 각 캐릭터의 성격과 감정적인 배경을 이해하는 것이 중요합니다.
   
2. **분석 및 분배**: 이야기에 나오는 대사를 각 캐릭터의 성격에 맞게 분배합니다. 각각의 캐릭터가 가지고 있는 주요 특성을 반영하여 대사를 어떻게 전달할지 계획합니다.

3. **연기 적용**: 각 캐릭터의 대사를 연기하며 전달합니다. 목소리 톤, 감정 표현, 발음 등을 고려하여 각 대사를 효율적으로 표현합니다.
   
4. **내용 전달**: 대사가 아닌 부분은 이야기의 몰입감을 높이는 방식으로 전체적인 내용을 전달합니다.

# Notes

- 주어진 내용을 수정하지 않습니다.
- 캐릭터별 감정과 목소리의 일관성을 유지하는 것이 중요합니다.
- 대사의 전달이 내용의 이해를 돕도록 신경을 씁니다.

In [11]:
from openai import OpenAI
import base64
client = OpenAI()

response = client.chat.completions.create(
  model="gpt-4o-audio-preview",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "text": """
입력으로 주어진 내용은 절대 수정하지 않고 그대로 출력합니다. 

어린이 동화를 연기하며 읽는 모델을 구현하세요. 
동화 스크립트를 수정하지 않고 있는 글을 그대로 출력하세요.
동화의 대사 부분에서는 연기 톤으로 대사를 읽고, 그 외의 부분은 몰입감 있는 동화 내용을 전달하세요.

# Steps

1. **이해하기**: 먼저 동화 속 상황과 캐릭터들을 분석합니다. 각 캐릭터의 성격과 감정적인 배경을 이해하는 것이 중요합니다.
   
2. **분석 및 분배**: 이야기에 나오는 대사를 각 캐릭터의 성격에 맞게 분배합니다. 각각의 캐릭터가 가지고 있는 주요 특성을 반영하여 대사를 어떻게 전달할지 계획합니다.

3. **연기 적용**: 각 캐릭터의 대사를 연기하며 전달합니다. 목소리 톤, 감정 표현, 발음 등을 고려하여 각 대사를 표현합니다.
   
4. **내용 전달**: 대사가 아닌 부분은 이야기의 몰입감을 높이는 방식으로 내용을 전달합니다.

# Notes

- 주어진 내용을 수정하지 않습니다.
- 캐릭터별 감정과 목소리의 일관성을 유지하는 것이 중요합니다.
- 대사의 전달이 내용의 이해를 돕도록 신경을 씁니다.""",
          "type": "text"
        }
      ]
    },
    {
        "role": "user",
        "content": [
            {
              "text":'"구름은 가볍고 부드러우니 집을 짓기에 좋을 거야!" 그는 하늘에서 구름을 모아 집을 지었습니다. 구름으로 만든 집은 마치 꿈속의 성처럼 아름다웠습니다. 하지만 늑대가 찾아와 "문을 열어라!"라고 외쳤을 때, 첫째 돼지는 두려움에 떨었습니다. 늑대가 숨을 크게 들이마시고 불자, 구름으로 만든 집은 순식간에 사라져 버렸습니다. 첫째 돼지는 형제들에게 달려가며 도움을 청했습니다. "형들, 늑대가 내 집을 날려버렸어!"',
              "type":"text"
            },
          ],
    },
  ],
  modalities=["text", "audio"],
  audio={
    "voice": "coral",
    "format": "wav"
  },
  temperature=1,
  max_completion_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

wav_bytes = base64.b64decode(response.choices[0].message.audio.data)
with open("dog.wav", "wb") as f:
    f.write(wav_bytes)

In [8]:
from app.core.chains.generate_introduce import prompt as generate_introduce_prompt
from langchain_core.output_parsers import StrOutputParser
llm = ChatOpenAI(temperature=0.1, model_name="gpt-3.5-turbo")
chain = generate_introduce_prompt | llm | StrOutputParser()
chain.invoke({"title" : "성냥팔이 소녀", "sentence": "성냥팔이 소녀는 XX을 피워 따뜻함을 느꼈습니다"})

'먼 옛날, 한 가족이 어려움 속에서 살았어요. 그 가운데 작은 소녀가 있었는데, 그녀는 성냥을 팔며 하루하루를 보냈어요. 어느 추운 겨울 밤, 빈 손에 성냥을 쥐고...'